Модифіковуємо зображення виділяючи контури


general purpose imports

In [4]:
import os
import cv2
import torch
import ultralytics

image modifications

In [ ]:
import os
import cv2

# src_dir = './RUSSIAN_PLANES_DB/train'
# src_dir = '../RUSSIAN_PLANAE_DB/test_10/'
# src_dir = '../RUSSIAN_PLANAE_DB/original/'
# dest_dir = '../RUSSIAN_PLANAE_DB/full_modified/'

src_dir =  "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\original_src"
dest_dir = "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\modified_src"


for filename in os.listdir(src_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        img_path = os.path.join(src_dir, filename)
        img = cv2.imread(img_path)
        print(img)
        cv2.imshow('image', img)
        print(os.path.exists(img_path))
        #CANNY
        # edges = cv2.Canny(img,150,250)

        #LAPLACE
        # LESS_CONTRAST_EDGES (ver 1)
        # denoise = cv2.GaussianBlur(img, (3,3), 0)
        # MORE_CONTRAST_EDGES (ver 2)
        denoise = cv2.GaussianBlur(img, (1,1), 0)
        
        #FURTHER_PROCESSING
        to_gray = cv2.cvtColor(denoise,  cv2.COLOR_BGR2GRAY)
        res_lap = cv2.Laplacian(to_gray, cv2.CV_16S, ksize=3)

        cv2.imwrite(os.path.join(dest_dir, filename), res_lap)
        # cv2.imwrite(os.path.join('./test_10_images/', filename), denoise) # last arg denoise or edges
        # cv2.imwrite(os.path.join('./test_10_images/', filename), to_gray)


Для розподілення датасету по train val папках


In [ ]:
import os
import cv2
import shutil


# Шлях до папки з зображеннями
src_folder =   "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\src\\modified_src"
train_folder = "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\images\\train"
val_folder =   "C:\\Users\\Северин\\Documents\\unik\\kursova\\DB\\images\\val"

# Отримати список всіх файлів у початковій папці
all_files = [f for f in os.listdir(src_folder) if os.path.isfile(os.path.join(src_folder, f))]

# Відсортувати файли (опціонально, залежить від вимог)
all_files.sort()

# Останні 50 файлів для валідаційного набору
val_files = all_files[-50:]
# Решта файлів для тренувального набору
train_files = all_files[:-50]

# Створити папки train та val, якщо вони ще не існують
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Переміщення файлів
for file_name in train_files:
    shutil.move(os.path.join(src_folder, file_name), os.path.join(train_folder, file_name))

for file_name in val_files:
    shutil.move(os.path.join(src_folder, file_name), os.path.join(val_folder, file_name))

print("Файли успішно розподілені між train та val папками.")


Тренування моделі

In [ ]:
# TRAINING & TEST MODEL
import torch
from ultralytics import YOLO
import os

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'avialable device: {device}')
torch.backends.cudnn.enabled = False

# Load a model
# project_folder = "C:\\Users\\Северин\\Documents\\unik\\kursova"
# model = YOLO('yolov8n.yaml')
# data = project_folder + 'src/CustomPlane_copy.yaml'

# load model
data = "CustomPlane.yaml"
model = YOLO('yolov8n.yaml')
model = YOLO('pts\\yolov8n.pt')

# Train and use the model with 527 images
results = model.train(data=data, epochs=200, imgsz=720, augment=True, device=0)
results = model.val()
results = model('..\\DB\\images\\train\\0266.png')

In [4]:
# import ultralytics
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model = YOLO('./pts/best_laplace.pt')

results = model.predict('../DB/images/train/0000.png')
# results = model.predict('../DB/original_images/0000.png')


for i, result in enumerate(results):
    boxes = result.boxes                # Boxes object for bounding box outputs
    masks = result.masks                # Masks object for segmentation masks outputs
    keypoints = result.keypoints        # Keypoints object for pose outputs
    probs = result.probs                # Probs object for classification outputs
    obb = result.obb                    # Oriented boxes object for OBB outputs
    result.show()                            # display to screen
    result.save(filename=f"../detections/result_{i}.jpg")  # save to disk


image 1/1 /home/sn/bardachok_local/univer/kursova/src/../DB/images/train/0000.png: 960x1024 7 Helicopters, 6 USSR|CIV/Transport/Recon.|AN-24/26/30s, 926.6ms
Speed: 20.3ms preprocess, 926.6ms inference, 3.5ms postprocess per image at shape (1, 3, 960, 1024)
0


In [1]:
test = model('..\\DB\\images\\train\\0266.png')
cv2.imshow("img", test)

NameError: name 'model' is not defined

In [ ]:
#IMAGE_PROCESSING
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')
model = YOLO('CustomPlane_copy.yaml')
model = YOLO('./pts/yolov8n.pt')

hyp = {
       'flipud': 0.5,
       'fliplr': 0.5,
       'rotate': 0.5,
       'mosaic': 0.3,
       'mixup':  0.05
}

# Train and use the model with 527 images
results = model.train(data='./CustomPlane_copy.yaml', epochs=500, imgsz=1000, hyp=hyp) 
results = model.val()
# results = model('../RUSSIAN_PLANAE_DB/original/0266.png')



In [ ]:
import cv2
# model = YOLO('D:/DesktopFolders/Універ/5курс/Сем_2/ООП/neuron/ultralytics-main/examples/best_laplace_contrast.pt')
model = YOLO('./pts/best_laplace_contrast.pt')
# image='D:/temp_2/0223.png'
image='../img_db/RUSSIAN_PLANAE_DB/0223.png'
img = cv2.imread(image)
class_name_dict = {
    0: 'Helicopter',
    1:'USSR|CIV/Transport|AN-124',
    2:'USSR|CIV/Transport/Recon.|AN-24/26/30',
    3:'USSR|Transport|AN-22',
    4:'USSR|Transport|AN-12',
    5:'USSR|CIV/Transport|AN-2',
    6:'USSR|Transport/Refuel.|IL-76/78',
    7:'USSR|Surv./MPA|IL-20/38',
    8:'CZECHOSLOVAKIA|Trainer/G-A|L-39',
    9:'RUSSIA|Trainer|YAK-130',
    10:'USSR|AWACS|A-50',
    11:'USSR|Bomber|TU-22',
    12:'USSR|Strat.Bomber|TU-22M',
    13:'USSR|Strat.Bomber/MAP|TU-95/142',
    14:'USSR|Strat.Bomber|TU-160',
    15:'USSR|Trainer|TU-134',
    16:'CIV AIRCRAFT',
    17:'USSR|Interceptor/G-A|MIG-31',
    18:'USSR|Fighter|MIG-29',
    19:'USSR|Mutlirole Fighter|SU-27/30/33/35',
    20:'USSR|Fighter/Bomber|SU-34',
    21:'USSR|CAS|SU-25',
    22:'USSR|Tact.Bomber|SU-24'
}
results = model(img)[0]   
for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(img, class_name_dict[int(class_id)].upper(), (int(x1), int(y1 - 10)),
            cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 1, cv2.LINE_AA)
            cv2.imwrite('recognized.jpg',img)

In [ ]:
# colab version of train


from ultralytics import YOLO
import os

# Load a model
# model = YOLO('yolov8n.yaml')
data = project_folder + 'src/CustomPlane_copy.yaml'

# load model
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

# Train and use the model with 527 images
results = model.train(data=data, epochs=100, imgsz=720, augment=True, batch=16)
results = model.val()
# results = model('../RUSSIAN_PLANAE_DB/original/0266.png')